Clone and set repo as current folder

In [ ]:
!git clone -b WGAN-GP-64 'https://github.com/leogenot/GenerativeDeepDrawing/'
%cd GenerativeDeepDrawing

Imports

In [ ]:
import os
import matplotlib.pyplot as plt

from models.WGANGP import WGANGP
from utils.loaders import load_dataset

import pickle
import tensorflow as tf

Run parameters

In [ ]:
# run params
SECTION = 'gan'
RUN_ID = '0003'
DATA_NAME = 'dataset64'
RUN_FOLDER = 'run/{}/'.format(SECTION)
RUN_FOLDER += '_'.join([RUN_ID, DATA_NAME])

if not os.path.exists(RUN_FOLDER):
    os.mkdir(RUN_FOLDER)
    os.mkdir(os.path.join(RUN_FOLDER, 'viz'))
    os.mkdir(os.path.join(RUN_FOLDER, 'images'))
    os.mkdir(os.path.join(RUN_FOLDER, 'weights'))

mode =  'load' #'load' to load old training or 'build' to create new training #

Training params

In [ ]:
BATCH_SIZE = 64
IMAGE_SIZE = 64

Load training

In [ ]:
x_train = load_dataset(DATA_NAME, IMAGE_SIZE, BATCH_SIZE)


Show image as array (as an example)

In [ ]:
x_train[0][0][0]

Show image

In [ ]:
plt.imshow((x_train[0][0][0]+1)/2)


WGAN parameters

In [ ]:
gan = WGANGP(input_dim = (IMAGE_SIZE,IMAGE_SIZE,3)
        , critic_conv_filters = [64,128,256,512]
        , critic_conv_kernel_size = [5,5,5,5]
        , critic_conv_strides = [2,2,2,2]
        , critic_batch_norm_momentum = None
        , critic_activation = 'leaky_relu'
        , critic_dropout_rate = None
        , critic_learning_rate = 0.0002
        , generator_initial_dense_layer_size = (4, 4, 512)
        , generator_upsample = [1,1,1,1]
        , generator_conv_filters = [256,128,64,3]
        , generator_conv_kernel_size = [5,5,5,5]
        , generator_conv_strides = [2,2,2,2]
        , generator_batch_norm_momentum = 0.9
        , generator_activation = 'leaky_relu'
        , generator_dropout_rate = None
        , generator_learning_rate = 0.0002
        , optimiser = 'adam'
        , grad_weight = 10
        , z_dim = 100
        , batch_size = BATCH_SIZE
        )

if mode == 'build':
    gan.save(RUN_FOLDER)

else:
    gan.load_weights(os.path.join(RUN_FOLDER, 'weights/weights.h5'))


Summary of critic parameters (saved in ./run/gan/FOLDER_NAME/viz

In [ ]:
gan.critic.summary()


Summary of generator parameters (saved in ./run/gan/FOLDER_NAME/viz

In [ ]:
gan.generator.summary()


Training parameters

In [ ]:
EPOCHS = 6501
PRINT_EVERY_N_BATCHES = 500
N_CRITIC = 5
BATCH_SIZE = 64


Actual training

In [ ]:
gan.train(     
    x_train
    , batch_size = BATCH_SIZE
    , epochs = EPOCHS
    , run_folder = RUN_FOLDER
    , print_every_n_batches = PRINT_EVERY_N_BATCHES
    , n_critic = N_CRITIC
    , using_generator = True
)

Plot of the training to see how it converged

In [ ]:
fig = plt.figure()
plt.plot([x[0] for x in gan.d_losses], color='black', linewidth=0.25)

plt.plot([x[1] for x in gan.d_losses], color='green', linewidth=0.25)
plt.plot([x[2] for x in gan.d_losses], color='red', linewidth=0.25)
plt.plot(gan.g_losses, color='orange', linewidth=0.25)

plt.xlabel('batch', fontsize=18)
plt.ylabel('loss', fontsize=16)

plt.xlim(0, 2000)
# plt.ylim(0, 2)

plt.show()
